<a href="https://colab.research.google.com/github/SvA1/Compling/blob/master/HW%203(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt

In [0]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt

In [0]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [0]:
import re
import gzip, csv
from string import punctuation
punctuation += "«»—…“”"
punct = set(punctuation)
from collections import Counter, defaultdict
from pprint import pprint
from nltk import sent_tokenize

!pip install symspellpy
from symspellpy import SymSpell, Verbosity
import pkg_resources

In [0]:
corpus = open('corpus_5000.txt', 'w')
with gzip.open('lenta-ru-news.csv.gz', 'rt') as archive:
    reader = csv.reader(archive, delimiter=',', quotechar='"')
    for i, line in enumerate(reader):
        if i < 5000: 
            corpus.write(line[2].replace('\xa0', ' ') + '\n')

In [0]:
def normalize(text):
    
    normalized_text = [(word.strip(punctuation)) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [0]:
import nltk
nltk.download('punkt')
corpus = []
for text in open('corpus_5000.txt').read().splitlines()[:12000]:
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [13]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
print(bad[1])
print(true[1])

Опофеозом дня для меня сегодня стала фраза услышанная в новостях:
Апофеозом дня для меня сегодня стала фраза услышанная в новостях


In [0]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [0]:
unigrams = Counter()
for sentence in corpus:
    unigrams.update(sentence)

In [0]:
def deletes(word):
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])     for i in range(len(word) + 1)]
    deletes1    = [L + R[1:]               for L, R in splits if R]
    splits2     = [(word[:i], word[i:])    for i in range(len(deletes1))]
    deletes2    = [L + R[2:]               for L, R in splits2 if R]
    deletes = deletes1 + deletes2
    return set(deletes)
    print("\n")

In [0]:
glob_dict = {}
for i in unigrams.keys():
  for j in deletes(i):
    glob_dict[j] = i

In [0]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [15]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


In [0]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], Sym_spell_func(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

In [21]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7020979020979021
0.5003837298541827
0.26771563110141267


In [0]:
def in_pro(input_word):
  candidates_list = []
  candidates_dict = {}
  strongest_candidate = input_word
  if input_word in unigrams:
    return input_word
  for i in deletes(input_word):
    if i in glob_dict:
      candidates_dict[unigrams[glob_dict[i]]] = glob_dict[i]
      strongest_candidate = max(candidates_dict.keys())
      strongest_candidate = candidates_dict[strongest_candidate]
  return strongest_candidate

In [42]:
in_pro('надаело')

'надоело'

In [0]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], in_pro(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

In [45]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.6594405594405595
0.36377590176515734
0.2963133111289767


### Триграммная модель

In [0]:
corpus_news = [['<start>', '<start>'] + sent + ['<end>'] for sent in corpus]

In [0]:
corpus = []
for text in open('corpus_5000.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [0]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [0]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in corpus_news:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))
    trigrams.update(ngrammer(sentence, n=3))

In [0]:
def trigram_in_pro(input_words):
  candidates_list = []
  candidates_dict = {}
  input_word = str(input_words.split()[-1])
  strongest_candidate = input_word
  if input_word in unigrams:
    return input_word
  for i in deletes(input_word):
    if i in glob_dict:
      one_word_prob = unigrams[glob_dict[i]]/len(unigrams)
      bigram = " ".join(input_words.split()[:-1]) 
      if unigrams[input_words.split()[0]]:
        bigram_prob = bigrams[bigram]/unigrams[input_words.split()[0]]
        trigram = bigram + " " + glob_dict[i]
      if bigrams[bigram]:
        trigram_prob = trigrams[trigram]/bigrams[bigram]
        prob_with_context = one_word_prob*(1 + trigram_prob/bigram_prob)
      else:
         prob_with_context = one_word_prob
      candidates_dict[unigrams[glob_dict[i]]] = glob_dict[i]
      strongest_candidate = max(candidates_dict.keys())
      strongest_candidate = candidates_dict[strongest_candidate]
  return strongest_candidate

In [65]:
trigram_in_pro("проходит со временим")

'временами'

In [0]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
  for j in range(len(true[i].split())):
    word_pairs = align_words(true[i], bad[i])
    if j == 0:
      r_bigram = "<start> <start> "
    elif j == len(true[i].split()):
      r_bigram = true[i][j-1] + " " + true[i][j] + " "
    else:
      r_bigram = true[i][j-2] + " " + true[i][j-1] + " "
    for pair in word_pairs:
      predicted = cashed.get(pair[1], trigram_in_pro(r_bigram + pair[1]))
      cashed[pair[0]] = predicted
      if predicted == pair[0]:
          correct += 1
      total += 1
      
      if pair[0] == pair[1]:
          total_correct += 1
          if pair[0] !=  predicted:
              correct_broken += 1
      else:
          total_mistaken += 1
          if pair[0] == predicted:
              mistaken_fixed += 1


In [68]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.6662871064269258
0.3399486633694359
0.2951235943410893
